<a href="https://colab.research.google.com/github/hansjang/Py_Radar/blob/main/%EB%B9%85%EB%B6%84%EA%B8%B0Part3_P198.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

(P.198) 2. 실습 A to Z : Penguins

---

**(1) 문제 : 펭귄의 무게를 예측하는 회귀 분석**

In [ ]:
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata

Cloning into 'yemoonsaBigdata'...
remote: Enumerating objects: 709, done.
remote: Counting objects: 100% (346/346), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 709 (delta 156), reused 237 (delta 115), pack-reused 363
Receiving objects: 100% (709/709), 11.46 MiB | 18.80 MiB/s, done.
Resolving deltas: 100% (341/341), done.


**(2) 사전 준비**

In [16]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

X_test = pd.read_csv("/content/yemoonsaBigdata/datasets/Part2/penguin_X_test.csv")
X_train = pd.read_csv("/content/yemoonsaBigdata/datasets/Part2/penguin_X_train.csv")
y_train = pd.read_csv("/content/yemoonsaBigdata/datasets/Part2/penguin_y_train.csv")

**(3) 데이터 전처리**

In [17]:
print(X_train.info())
print(X_test.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            240 non-null    object 
 1   island             240 non-null    object 
 2   sex                232 non-null    object 
 3   bill_length_mm     238 non-null    float64
 4   bill_depth_mm      238 non-null    float64
 5   flipper_length_mm  238 non-null    float64
dtypes: float64(3), object(3)
memory usage: 11.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            101 non-null    object 
 1   island             101 non-null    object 
 2   sex                101 non-null    object 
 3   bill_length_mm     101 non-null    float64
 4   bill_depth_mm      101 non-null    float64
 5   flipper_l

**[결측치 처리]** X_train 및 X_test에는 6개의 컬럼이 있으며, 결측치의 경우 X_train에서 sex 변수에 8개, bill_length_mm, bill_depth_mm, flipper_length_mm 에 각각 2개씩 존재한다. y_train에는 2개의 결측치가 있다. species, island, sex는 명목형 변수이며 bill_length_mm, bill_depth_mm, flipper_length_mm, body_mass_g 는 수치형 변수이다.

In [18]:
train = pd.concat([X_train, y_train], axis = 1)
print(train.loc[(train.sex.isna()) | (train.bill_length_mm.isna()) | (train.bill_depth_mm.isna()) | (train.flipper_length_mm.isna()) |
 (train.body_mass_g.isna())])

    species     island  sex  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0    Adelie  Torgersen  NaN            42.0           20.2              190.0   
6    Gentoo     Biscoe  NaN            44.5           14.3              216.0   
43   Gentoo     Biscoe  NaN             NaN            NaN                NaN   
66   Adelie  Torgersen  NaN            37.8           17.3              180.0   
88   Gentoo     Biscoe  NaN            47.3           13.8              216.0   
89   Adelie  Torgersen  NaN            37.8           17.1              186.0   
110  Gentoo     Biscoe  NaN            44.5           15.7              217.0   
229  Adelie  Torgersen  NaN             NaN            NaN                NaN   

     body_mass_g  
0         4250.0  
6         4100.0  
43           NaN  
66        3700.0  
88        4725.0  
89        3300.0  
110       4875.0  
229          NaN  


isna 함수를 사용하여 결측치가 하나라도 존재하는 관측값들을 확인한 결과, sex가 모두 결측치로 나타난다. sex는 대치하기가 곤란한 명목형 변수이므로 결측치를 포함하는 8개의 관측치를 제거하는 방향으로 전처리한다.

In [19]:
train = train.dropna()
train.reset_index(drop=True, inplace=True)

In [20]:
X_train = train[['species', 'island', 'sex', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm']]
y_train = train[['body_mass_g']]

In [21]:
print(X_train.describe())

       bill_length_mm  bill_depth_mm  flipper_length_mm
count      232.000000     232.000000         232.000000
mean        43.990948      17.226293         200.681034
std          5.509760       1.964677          14.064231
min         32.100000      13.200000         172.000000
25%         39.200000      15.700000         190.000000
50%         44.950000      17.350000         197.000000
75%         48.775000      18.725000         212.250000
max         58.000000      21.500000         231.000000


In [22]:
COL_DEL = []
COL_NUM = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm']
COL_CAT = ['species', 'island', 'sex']
COL_Y = ['body_mass_g']

수치형 변수(COL_NUM)는 스케일링을 하고, 범주형 변수(COL_CAT)는 레이블 인코딩을 실시한다.

In [23]:
X = pd.concat([X_train, X_test])

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown = 'ignore')
ohe.fit(X[COL_CAT])

X_train_res = ohe.transform(X_train[COL_CAT])
X_test_res = ohe.transform(X_test[COL_CAT])

In [ ]:
print(X_train_res)

In [26]:
X_train_ohe = pd.DataFrame(X_train_res.todense(), columns = ohe.get_feature_names_out())
X_test_ohe = pd.DataFrame(X_test_res.todense(), columns = ohe.get_feature_names_out())
print(X_train_ohe)

X_train_fin = pd.concat([X_train[COL_NUM], X_train_ohe], axis = 1)
X_test_fin = pd.concat([X_test[COL_NUM], X_test_ohe], axis = 1)

     species_Adelie  species_Chinstrap  species_Gentoo  island_Biscoe  \
0               0.0                0.0             1.0            1.0   
1               1.0                0.0             0.0            0.0   
2               0.0                1.0             0.0            0.0   
3               0.0                0.0             1.0            1.0   
4               1.0                0.0             0.0            0.0   
..              ...                ...             ...            ...   
227             0.0                1.0             0.0            0.0   
228             0.0                0.0             1.0            1.0   
229             1.0                0.0             0.0            0.0   
230             0.0                1.0             0.0            0.0   
231             0.0                0.0             1.0            1.0   

     island_Dream  island_Torgersen  sex_FEMALE  sex_MALE  
0             0.0               0.0         1.0       0.0  
1  

원-핫 인코딩을 적용한 후의 희소 행렬(sparse matrix)을 밀집 행렬(dense matrix)로 변환하고, 데이터프레임으로 변환하여 원래 열 이름을 이용해 새로운 데이터프레임을 생성한다. 이후 concat을 이용하여 수치형 변수 집합과 결합한다.

**(4) 데이터 모형 구축**

In [27]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train_fin, y_train, test_size = 0.3)

In [28]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_tr[COL_NUM])
X_tr[COL_NUM] = scaler.transform(X_tr[COL_NUM])
X_val[COL_NUM] = scaler.transform(X_val[COL_NUM])
X_test_fin[COL_NUM] = scaler.transform(X_test_fin[COL_NUM])

학습데이터와 검증데이터가 7:3으로 분리되었으니 수치형 변수에 대한 스케일링을 진행한다. MinMaxScaler를 사용하였다.

In [ ]:
from sklearn.linear_model import LinearRegression

modelLR = LinearRegression()
modelLR.fit(X_tr, y_tr)

y_val_pred = modelLR.predict(X_val)
print(y_val_pred)

In [31]:
print(modelLR.intercept_)

coef = pd.Series(data = modelLR.coef_[0], index = X_train_fin.columns)
print(coef.sort_values())

[3332.97935061]
species_Chinstrap   -518.142221
sex_FEMALE          -208.567376
species_Adelie      -161.872064
island_Dream         -42.506563
island_Biscoe        -12.192105
island_Torgersen      54.698668
sex_MALE             208.567376
bill_depth_mm        260.905109
species_Gentoo       680.014285
flipper_length_mm    713.566616
bill_length_mm       735.472241
dtype: float64


intercept_ 와 coef_ 를 사용하여 절편과 기울기를 구할 수 있다. 펭귄의 종이 "Chinstrap"이나 "Adelie"일 경우 무게에 음의 영향을 주고 있고, "Gentoo"일 경우 무에게 양의 영향을 주고 있다. 성별의 경우 수컷이 암컷보다 무겁고, 부리의 크기와 날개의 길이가 클 수록 무게도 많이 나간다는 것을 알 수 있다.

**(6) 데이터 모형 평가**

In [36]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_val, y_val_pred)
rmse = mean_squared_error(y_val, y_val_pred, squared = False)

print('MSE :', round(mse, 2))
print('RMSE :', round(rmse, 2))

MSE : 81264.88
RMSE : 285.07


In [ ]:
y_pred = modelLR.predict(X_test_fin)
print(y_pred)

In [42]:
pd.DataFrame({'body_mass_g': y_pred[: ,0]}).to_csv('./penguin_y_test.csv', index=False)